In [1]:

# coding: utf-8


import simuOpt
simuOpt.setOptions(alleleType='lineage', optimized=True, quiet=True)
import simuPOP as sim
import pandas as pd
import collections as col
from wgs import breed, operators, selection, helpers, parser, parameterizer, selection
import random
import numpy as np
random.seed(1337)


hapmap = pd.read_csv('clean_hapmap.txt')
genetic_map = hapmap.ix[:, :'cM_pos']
genetic_map = pd.read_csv('nam_prefounders_genetic_map.txt', index_col=None,
                         sep='\t')

chr_cm_positions = col.OrderedDict()
for i in range(1, 11):
    chr_cm_positions[i] = []

for idx in range(len(genetic_map)):
    chr_cm_positions[int(genetic_map.iloc[idx]['chr'])].append(
    float(genetic_map.iloc[idx]['cM_pos']))


cM_positions = []
for k, v in chr_cm_positions.items():
    cM_positions.append(v)


snp_to_integer = {'A':0, 'C':1, 'G':2, 'T':3, '-':4, '+':5}

integral_valued_loci = []
relative_integral_valued_loci = {}
for idx in range(len(genetic_map)):
    if str(genetic_map.iloc[idx]['cM_pos'])[-2:] == '.0':
        integral_valued_loci.append(idx)
        relative_integral_valued_loci[idx] = (genetic_map.iloc[idx]['chr'], genetic_map.iloc[idx]['cM_pos'])


qtl = sorted(random.sample(integral_valued_loci, 5))

triplet_qtl = []
for locus in qtl:
    triplet_qtl.append(locus-1)
    triplet_qtl.append(locus)
    triplet_qtl.append(locus+1)
triplet_qtl = sorted(triplet_qtl)


alleles = {i: (snp_to_integer[hapmap.ix[i, 'alleles'][0]], 
               snp_to_integer[hapmap.ix[i, 'alleles'][-1]]) for i in
          range(len(hapmap))}


allele_effects = {}
for tqtl in triplet_qtl:
    for allele in alleles[tqtl]:
        allele_effects[tqtl, allele] = random.expovariate(1)


recombination_rates = []
for chromosome in cM_positions:
    for cM in chromosome:
        if str(cM)[-2:] == '.6':
            recombination_rates.append(0.01)
        else:
            recombination_rates.append(0.0)

allele_names = ['A', 'C', 'T', 'G', 'D', 'I']

flat_cM_positions = []
for cMs in cM_positions:
    flat_cM_positions.extend(cMs)


nam = sim.loadPopulation('nam_prefounders.pop')
nam.setSubPopName('prefounders', 0)
sample_sizes = {i: 100 for i in range(0, 11, 2)}

s = selection.Truncation(10,
                       3,
                       2000,
                       0.05,
                       0.50,
                       5,
                       0.7,
                       sample_sizes,
                       1)

nam.dvars().qtl = qtl
nam.dvars().triplet_qtl = triplet_qtl
nam.dvars().allele_effects = allele_effects
sim.tagID(nam, reset=True)

founders = [(1, 5), (7, 8)]
replicated_nam = sim.Simulator(nam, rep=2)
pop = replicated_nam.extract(0)
meta = replicated_nam.extract(0)
meta.removeSubPops(0)
s.generate_f_one(pop, recombination_rates, founders)
s.generate_f_two(pop, recombination_rates)
s.interim_random_mating(pop, recombination_rates)
s.mate_and_merge(pop, recombination_rates)
s.recurrent_truncation_selection(pop, meta, triplet_qtl, allele_effects,
                                recombination_rates)
                                


Creating the F_one population from selected founders.
Generation: 0
Creating the F_two population.
Generation: 1
Initiating interim random mating for 3 generations.
Generation: 2
Generation: 3
Generation: 4
Initiating recombinatorial convergence at generation: 5
Processing 1 replications...
Initiating recurrent selection for 10 generations.
Initial: Sampled 100 individuals from generation 0 Replicate: 0.
Generation: 0
Generation: 1
Generation: 2
Generation: 3
Generation: 4
Generation: 5
Generation: 6
Generation: 7
Generation: 8
Generation: 9
Final: Sampled 100 individuals from generation 10


In [13]:
"""

pop.save('testpop_recurrent_selection.pop')
meta.save("test_metapop_recurrent_selection.pop")

"""